<a href="https://colab.research.google.com/github/rishank012/Capstone-Project/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathway bokeh panel --quiet

In [ ]:
import pandas as pd
import pathway as pw
import bokeh.plotting
from bokeh.models import LinearAxis, Range1d, HoverTool
from bokeh.transform import linear_cmap
import panel as pn
import os

# Step 1: Importing and Preprocessing the Data

In [ ]:
    from google.colab import files
    uploaded = files.upload()

In [ ]:
df = pd.read_csv('dataset.csv')
df

In [ ]:
to_num = ['ID','Occupancy','Capacity','QueueLength','TrafficConditionNearby','IsSpecialDay']
df[to_num] = (
    df[to_num]
      .apply(lambda c: pd.to_numeric(c, errors='coerce'))
      .fillna(0)
)

In [ ]:
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)
df = df.sort_values(['ID','Timestamp']).reset_index(drop=True)

# Feature Engineering:

In [ ]:
df['occ_rate'] = df['Occupancy'] / df['Capacity']

for col in ['QueueLength','TrafficConditionNearby']:
    mn = df.groupby('ID')[col].transform('min')
    mx = df.groupby('ID')[col].transform('max')
    df[f'{col}_norm'] = (df[col] - mn) / (mx - mn + 1e-6)

vt_map = {'car':1.0,'bike':0.5,'truck':1.5}
df['VT_weight'] = df['VehicleType'].map(vt_map).fillna(1.0)

### **Model 1:** *Baseline Linear Model*

In [ ]:
alpha = 0.5
df['delta1'] = alpha * df['occ_rate']
df['price_m1'] = df.groupby('ID')['delta1'].cumsum().add(10)

### **Model 2:** *Demand Based Price Function*

In [ ]:
a,b,c,d,e = 1.0,0.8,0.6,1.2,0.5
lam = 0.8

df['raw_d'] = (
      a*df['occ_rate']
    + b*df['QueueLength_norm']
    - c*df['TrafficConditionNearby_norm']
    + d*df['IsSpecialDay']
    + e*df['VT_weight']
)
dmin = df.groupby('ID')['raw_d'].transform('min')
dmax = df.groupby('ID')['raw_d'].transform('max')
df['d_norm'] = (df['raw_d'] - dmin) / (dmax - dmin + 1e-6)

df['price_m2'] = 10*(1 + lam*df['d_norm'])
df['price_m2'] = df['price_m2'].clip(5,20)

###**Model 3:** *Competitive Pricing Model*

In [ ]:
def comp_logic(r):
    if r['Occupancy'] >= r['Capacity']:
        r['price_m3'] = r['price_m2'] * 0.7
        r['suggest_reroute'] = True
    else:
        uplift = 0.1 * r['price_m2'] * (1 - r['TrafficConditionNearby_norm'])
        r['price_m3'] = r['price_m2'] + uplift
        r['suggest_reroute'] = False
    return r

df = df.apply(comp_logic, axis=1)

In [ ]:
for m in ['price_m1','price_m2','price_m3']:
    df[f'{m}_sm'] = df.groupby('ID')[m].transform(lambda x: x.ewm(alpha=0.2).mean())

In [ ]:
out = [
  'Timestamp','ID','Occupancy','Capacity','occ_rate',
  'QueueLength_norm','TrafficConditionNearby_norm','IsSpecialDay',
  'VT_weight','price_m1','price_m2','price_m3',
  'price_m1_sm','price_m2_sm','price_m3_sm','suggest_reroute'
]
df[out].to_csv('parking_stream_final.csv', index=False)

### **Defining Pathway Schema:**

In [ ]:
class ParkingSchema(pw.Schema):
    Timestamp: str; ID: int; Occupancy: int; Capacity: int
    occ_rate: float; QueueLength_norm: float; TrafficConditionNearby_norm: float
    IsSpecialDay: int; VT_weight: float
    price_m1: float; price_m2: float; price_m3: float
    price_m1_sm: float; price_m2_sm: float; price_m3_sm: float
    suggest_reroute: bool

stream = pw.demo.replay_csv(
    'parking_stream_final.csv',
    schema=ParkingSchema,
    input_rate=500
).with_columns(
    t = pw.this.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S")
)

## **Bokeh visualization of all three models:**

In [21]:
def enhanced_plot(src):
    fig = bokeh.plotting.figure(
        height=450, width=950,
        title="Dynamic Pricing Models w/ Occupancy Overlay",
        x_axis_type="datetime"
    )
    # Smooth prices
    fig.line('t','price_m1_sm', source=src, color='blue',  line_width=2, legend_label="Model 1 (EWMA)")
    fig.line('t','price_m2_sm', source=src, color='green', line_width=2, legend_label="Model 2 (EWMA)")
    fig.line('t','price_m3_sm', source=src, color='red',   line_width=2, legend_label="Model 3 (EWMA)")

    # Occupancy on right axis
    fig.extra_y_ranges = {"occ": Range1d(0,1)}
    fig.add_layout(LinearAxis(y_range_name="occ", axis_label="Occupancy Rate"), 'right')
    fig.line('t','occ_rate', source=src, y_range_name="occ",
             color='orange', line_dash='dotted', line_width=2,
             legend_label="Occupancy Rate")

    # Reroute highlight via color map (gray=false, black=true)
    cmap = linear_cmap('suggest_reroute', ['lightgray','black'], 0,1)
    fig.scatter('t','price_m3', source=src, size=6, color=cmap, legend_label="Reroute Flag")

    # Hover
    hover = HoverTool(tooltips=[
        ("Time", "@t{%F %H:%M}"),
        ("Model 1", "@price_m1{0.00}"),
        ("Model 2", "@price_m2{0.00}"),
        ("Model 3", "@price_m3{0.00}"),
        ("Occ", "@occ_rate{0.0%}")
    ], formatters={'@t':'datetime'}, mode='vline')
    fig.add_tools(hover)

    fig.legend.location = "top_left"
    fig.legend.click_policy = "hide"
    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price (USD)"

    return fig

pn.extension()
dashboard = pn.Column(stream.plot(enhanced_plot, sorting_col='t'))
dashboard.save('dashboard.html', embed=True)

dashboard.servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [22]:
os.environ['PW_RUN_SANDBOX'] = 'true'
pw.run()

Output()